In [1]:
from sympy import *
from strawberryfields.decompositions import bloch_messiah
from sympy.physics.quantum.dagger import Dagger
import numpy as np
import scipy as sp
init_printing(use_latex="mathjax", latex_mode="equation")
np.set_printoptions(precision=3,suppress=True)

In [36]:
# sym_func = lambdify(s,  Q*S*Q.T, modules='numpy')
# def decomp(l):
#     O1, D, O2 = bloch_messiah(sym_func(l))
#     return D,O1,O2
# print(decomp(2)[0])
# sigma = simplify(expand((S*S.T)**1/2))
# sigma
# O,D = sigma.diagonalize()


In [2]:
yvar = symbols('y1:5')
tvar = symbols('t1:9')
rvar = symbols('r1:11')
x = Matrix([[1,0],[0,1]])
y = Matrix([[yvar[0],yvar[1]],[yvar[2],yvar[3]]])
t = Matrix([[tvar[0],tvar[1],tvar[2],tvar[3]],[tvar[4],tvar[5],tvar[6],tvar[7]]])
r = Matrix([
    [rvar[0],rvar[1],rvar[2],rvar[3]],
    [rvar[1],rvar[4],rvar[5],rvar[6]],
    [rvar[2],rvar[5],rvar[7],rvar[8]],
    [rvar[3],rvar[6],rvar[8],rvar[9]]])
omega1 = Matrix([[0,1],[-1,0]])
omega2 = Matrix([[omega1,zeros(2)],[zeros(2),omega1]])
def cond0():
    return t*omega2*t.T
def cond1():
    return t*t.T - y
def cond2():
    return t.T*omega1 + r*omega2*t.T
def cond3():
    return t.T*omega1*t + r*omega2*r.T - omega2

def tconds():
    return Matrix([[cond0(),cond1()]])


In [3]:
s = Symbol("s", positive = True)
def allconds():
    return Matrix([[tconds()],[cond2().T],[cond3()]])
simplify(expand(allconds()).subs([
    (yvar[0],2*s*s),
    (yvar[1],0),
    (yvar[2],0),
    (yvar[3],2*s*s),
    (tvar[0],0),
    (tvar[1],0),
    (tvar[2],sqrt(2)*s),
    (tvar[3],0),
    (tvar[4],0),
    (tvar[5],sqrt(2)*s),
    (tvar[6],0),
    (tvar[7],0),
    (rvar[0],0),
    (rvar[1],0),
    (rvar[2],-1),
    (rvar[3],0),
    (rvar[4],s*s),
    (rvar[5],0),
    (rvar[6],-1),
    (rvar[7],s*s),
    (rvar[8],0),
    (rvar[9],0),
    ]))

⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

In [4]:
tparam = simplify(expand(t.subs([
    (tvar[0],0),
    (tvar[1],0),
    (tvar[2],sqrt(2)*s),
    (tvar[3],0),
    (tvar[4],0),
    (tvar[5],sqrt(2)*s),
    (tvar[6],0),
    (tvar[7],0),
])))

yparam = simplify(expand(y.subs([
    (yvar[0],2*s*s),
    (yvar[1],0),
    (yvar[2],0),
    (yvar[3],2*s*s)
])))

rparam = simplify(expand(r.subs([
    (rvar[0],0),
    (rvar[1],0),
    (rvar[2],-1),
    (rvar[3],0),
    (rvar[4],s*s),
    (rvar[5],0),
    (rvar[6],-1),
    (rvar[7],s*s),
    (rvar[8],0),
    (rvar[9],0),
    ])))


sparam = Matrix([
    [eye(2),tparam],[tparam.T,rparam]
])
sparam

⎡ 1     0    0    0    √2⋅s  0 ⎤
⎢                              ⎥
⎢ 0     1    0   √2⋅s   0    0 ⎥
⎢                              ⎥
⎢ 0     0    0    0     -1   0 ⎥
⎢                              ⎥
⎢                  2           ⎥
⎢ 0    √2⋅s  0    s     0    -1⎥
⎢                              ⎥
⎢                        2     ⎥
⎢√2⋅s   0    -1   0     s    0 ⎥
⎢                              ⎥
⎣ 0     0    0    -1    0    0 ⎦

In [6]:
commutation = Matrix([[omega1,zeros(2),zeros(2)],[zeros(4,2),omega2]])
commutation2 = Matrix([
    [0,0,0,1,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,1],
    [-1,0,0,0,0,0],
    [0,-1,0,0,0,0],
    [0,0,-1,0,0,0]
])
Q = Matrix([
    [0,0,0,0,1,0],
    [0,0,0,1,0,0],
    [1,0,0,0,0,0],
    [0,0,0,0,0,1],
    [0,0,-1,0,0,0],
    [0,1,0,0,0,0]
])

V = Matrix([
    [1, 1j, 0, 0, 0, 0],
    [0, 0, 1, 1j, 0, 0],
    [0, 0, 0, 0, 1, 1j]
])
Pi = Matrix([
    [1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1]
])
def isSymplectic(m,c):
    return simplify(expand(m*c*m.T-c))==zeros(6)
def correctVariance(s,v,y):
    return simplify(expand((s*v*s.T)[:2,:2]-y) == zeros(2))
b1= isSymplectic(sparam,commutation)
b2 =correctVariance(sparam,np.eye(6),yparam+np.eye(2))
b1,b2


(True, True)

In [8]:
mparam = Pi * sparam * Pi.T
mparam

⎡ 1    0   √2⋅s   0     0    0 ⎤
⎢                              ⎥
⎢ 0    0    -1    0     0    0 ⎥
⎢                              ⎥
⎢            2                 ⎥
⎢√2⋅s  -1   s     0     0    0 ⎥
⎢                              ⎥
⎢ 0    0    0     1    √2⋅s  0 ⎥
⎢                              ⎥
⎢                        2     ⎥
⎢ 0    0    0    √2⋅s   s    -1⎥
⎢                              ⎥
⎣ 0    0    0     0     -1   0 ⎦

In [19]:
block1 = mparam[:3,:3]
block2 = mparam[:3,3:]
block3 = mparam[3:,:3]
block4 = mparam[3:,3:]

# A =1/2* (block1 + block4 + 1j*(block3-block2))
# B =1/2* (block1 - block4 + 1j*(block2+block3))

# Bogo = Matrix([[A,B],[B.conjugate(),A.conjugate()]])
# bogo_func = lambdify(s, Bogo, modules='numpy')
# bvalued = bogo_func(1)
# bvalued
block1*block4

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

In [17]:
w,v = np.linalg.eigh(bvalued)
w

array([-1.   , -1.   ,  0.382,  0.382,  2.618,  2.618])

In [18]:
kvalued = sp.linalg.logm(bvalued)
kvalued

array([[-0.   +0.628j, -0.   -0.889j, -0.   -0.889j,  0.   -0.j   ,
        -0.609+0.j   ,  0.609+0.j   ],
       [-0.   -0.889j, -0.   +1.257j, -0.   +1.257j, -0.609-0.j   ,
        -0.43 +0.j   , -0.   +0.j   ],
       [-0.   -0.889j, -0.   +1.257j, -0.   +1.257j,  0.609-0.j   ,
         0.   +0.j   ,  0.43 +0.j   ],
       [ 0.   +0.j   , -0.609-0.j   ,  0.609-0.j   , -0.   +0.628j,
        -0.   -0.889j, -0.   -0.889j],
       [-0.609+0.j   , -0.43 -0.j   , -0.   +0.j   , -0.   -0.889j,
        -0.   +1.257j,  0.   +1.257j],
       [ 0.609-0.j   , -0.   +0.j   ,  0.43 +0.j   , -0.   -0.889j,
         0.   +1.257j, -0.   +1.257j]])

In [72]:
u1,d1=block1.diagonalize()
u1 = simplify(factor(u1))
d1 = simplify(expand(d1))
# u1_inv =simplify(expand(u1**(-1)))
d1

⎡-1            0                       0           ⎤
⎢                                                  ⎥
⎢              ________                            ⎥
⎢     2       ╱  2                                 ⎥
⎢    s    s⋅╲╱  s  + 4                             ⎥
⎢0   ── - ───────────── + 1            0           ⎥
⎢    2          2                                  ⎥
⎢                                                  ⎥
⎢                                      ________    ⎥
⎢                             2       ╱  2         ⎥
⎢                            s    s⋅╲╱  s  + 4     ⎥
⎢0             0             ── + ───────────── + 1⎥
⎣                            2          2          ⎦

In [97]:
Um = 1/(sqrt(s**2+4))*Matrix([
    [s,-sqrt(2),-sqrt(2)],
    [-sqrt(2), -1/2*(s+sqrt(s**2 + 4)),  -1/2*(s-sqrt(s**2 + 4))],
    [-sqrt(2), -1/2*(s-sqrt(s**2 + 4)),  -1/2*(s+sqrt(s**2 + 4))]
])
rm = Matrix([
    [-1,0,0],
    [0,( -1/2*(s-sqrt(s**2 + 4)))**2,0],
    [0,0,(-1/2*(s+sqrt(s**2 + 4)))**2]
])
logrm = Matrix([
    [1j*pi,0,0],
    [0,2*ln( -1/2*(s-sqrt(s**2 + 4))),0],
    [0,0,2*ln(1/2*(s+sqrt(s**2 + 4)))]
])
# logrm
# simplify(expand((Um@logrm@Um.T)))

xi = (Um@logrm@Um.T).subs(s,1).evalf()
xi

⎡      0.e-21 + 0.628318530717959⋅ⅈ        -0.608690161677935 - 0.888576587631
⎢                                                                             
⎢-0.608690161677935 - 0.888576587631673⋅ⅈ  -0.430408940964004 + 1.256637061435
⎢                                                                             
⎣0.608690161677935 - 0.888576587631673⋅ⅈ         0.e-21 + 1.25663706143592⋅ⅈ  

673⋅ⅈ  0.608690161677935 - 0.888576587631673⋅ⅈ⎤
                                              ⎥
92⋅ⅈ         0.e-21 + 1.25663706143592⋅ⅈ      ⎥
                                              ⎥
       0.430408940964004 + 1.25663706143592⋅ⅈ ⎦

In [95]:
logrm.subs(s,1).evalf()

⎡3.14159265358979⋅ⅈ          0                             0                  
⎢                                                                             
⎢        0           -0.962423650119207                    0                  
⎢                                                                             
⎣        0                   0           0.962423650119207 + 6.28318530717959⋅

 ⎤
 ⎥
 ⎥
 ⎥
ⅈ⎦

In [229]:
(block1*block1.T)**(1/2)

⎡                                                                             
⎢                                                                 ⎛           
⎢         ⎛                   ________                  ________⎞ ⎜ 4    3   ╱
⎢         ⎜      3       2   ╱  2                      ╱  2     ⎟ ⎜s    s ⋅╲╱ 
⎢   2     ⎝- √2⋅s  + √2⋅s ⋅╲╱  s  + 4  - 3⋅√2⋅s + √2⋅╲╱  s  + 4 ⎠⋅⎜── - ──────
⎢  s                                                              ⎝2          
⎢────── - ────────────────────────────────────────────────────────────────────
⎢ 2                                     ⎛           ________                 _
⎢s  + 4                                 ⎜ 4    3   ╱  2           2         ╱ 
⎢                                       ⎝s  - s ⋅╲╱  s  + 4  + 4⋅s  - 2⋅s⋅╲╱  
⎢                                                                             
⎢                                                                             
⎢                                                   

In [203]:
lamda = symbols('lamda')
ap = a.charpoly(lamda)
apoly = factor(simplify(expand(ap.as_expr())))
solve(apoly)

⎡           ⎧        2      ⎫⎤
⎢{λ: -1.0}, ⎨λ: 0.5⋅s  + 1.0⎬⎥
⎣           ⎩               ⎭⎦

In [204]:
a.diagonalize()

⎛⎡                      1.4142135623731  1.4142135623731⎤  ⎡-1.0       0      
⎜⎢-0.707106781186548⋅s  ───────────────  ───────────────⎥  ⎢                  
⎜⎢                             s                s       ⎥  ⎢           2      
⎜⎢                                                      ⎥, ⎢ 0    0.5⋅s  + 1.0
⎜⎢        1.0                 1.0               0       ⎥  ⎢                  
⎜⎢                                                      ⎥  ⎢                  
⎝⎣        1.0                  0               1.0      ⎦  ⎣ 0         0      

       0      ⎤⎞
              ⎥⎟
              ⎥⎟
       0      ⎥⎟
              ⎥⎟
       2      ⎥⎟
  0.5⋅s  + 1.0⎦⎠

In [ ]:
# sval = sparam.subs(s,1)
# E = simplify(expand(V*sval*Dagger(V)/2))
# F = simplify(expand(V*sval*V.T/2))
# Fval = np.array(F).astype(np.cdouble)
# Eval = np.array(E).astype(np.cdouble)
# fun,fs,fvh = np.linalg.svd(Fval)
# eun,es,evh = np.linalg.svd(Eval)

In [69]:
R = np.diag(np.arcsinh(fs))
# eR = np.diag(np.arccosh(es))
eith = np.conj(fvh.T)
phi = -1j* sp.linalg.logm(fun)
xi = np.matmul(R,eith)
(xi,phi)

(array([[-0.478+0.j   ,  0.597+0.j   , -0.266+0.j   ],
        [ 0.   -0.193j,  0.   -0.348j,  0.   -0.434j],
        [ 0.   -0.163j,  0.   -0.072j,  0.   +0.13j ]]),
 array([[ 0.279+0.j   , -0.573-0.465j, -0.465-0.573j],
        [-0.573+0.465j,  1.431+0.j   , -0.145+0.66j ],
        [-0.465+0.573j, -0.145-0.66j ,  1.431-0.j   ]]))

In [55]:
(eith,R)

(array([[-0.59100905-0.j        ,  0.73697623-0.j        ,
         -0.32798528-0.j        ],
        [ 0.        -0.32798528j,  0.        -0.59100905j,
          0.        -0.73697623j],
        [ 0.        -0.73697623j,  0.        -0.32798528j,
          0.        +0.59100905j]]),
 array([0.80958692, 0.58886261, 0.22072431]))

In [42]:
lamda = symbols('lamda')
sp = svalued.charpoly(lamda)
spoly = factor(simplify(expand(sp.as_expr())))
solve(spoly)[0],spoly

NameError: name 'svalued' is not defined

In [ ]:
co1,co2,co3,co4,co5,co6= symbols('co1:7')
n1,n2 = symbols('n1:3')
d1,d2 = symbols('d1:3')
f1,f2 = symbols('f1:3')
r1,r2 = symbols('r1:3')
co1= (16-9*s*s)/2
co2 = (3*s**2 + 4)
co3 = simplify(expand(sqrt(co2**3-co1**2)))
co4 = co1 + I* co3
co5 = co1 - I* co3
co6 = simplify(cbrt(factor(expand(co4*co5))))
n1 = -co2*cbrt(co5)
d1 = 3*co6
n2 = -cbrt(co4)
d2 = 3
f1= simplify(n1/d1) 
f2= simplify(n2/d2)
f1 + f2


           _______________________________________________            ________
          ╱                      ___________________                 ╱        
   2/3 3 ╱       2              ╱    4       2                2/3 3 ╱       2 
  2   ⋅╲╱   - 9⋅s  - 3⋅√3⋅ⅈ⋅s⋅╲╱  4⋅s  + 13⋅s  + 32  + 16    2   ⋅╲╱   - 9⋅s  
- ──────────────────────────────────────────────────────── - ─────────────────
                             6                                                

_______________________________________
              ___________________      
             ╱    4       2            
+ 3⋅√3⋅ⅈ⋅s⋅╲╱  4⋅s  + 13⋅s  + 32  + 16 
───────────────────────────────────────
          6                            

In [ ]:
simplify(factor(expand(svalued.singular_values()[0])))

             _________________________________________________________________
            ╱                                     ____________________________
           ╱                                     ╱                            
          ╱     3 ___  7      3 ___  5      4 3 ╱       9       7       5     
         ╱    4⋅╲╱ 2 ⋅s  + 16⋅╲╱ 2 ⋅s  + 2⋅s ⋅╲╱   - 2⋅s  - 12⋅s  - 42⋅s  - 70
√6⋅     ╱     ────────────────────────────────────────────────────────────────
       ╱                                                                      
      ╱                                                                       
     ╱                                                                        
   ╲╱                                                                         
──────────────────────────────────────────────────────────────────────────────
                                                                              

___________________________________________________

In [ ]:
E =simplify(expand(V*svalued*Dagger(V)/2))
F = simplify(expand(V*svalued*V.T/2))
(E,F)

⎛⎡       -ⅈ⋅s   ⅈ⋅s⎤  ⎡     ⅈ⋅s   ⅈ⋅s⎤⎞
⎜⎢  1    ─────  ───⎥  ⎢ 0   ───   ───⎥⎟
⎜⎢         2     2 ⎥  ⎢      2     2 ⎥⎟
⎜⎢                 ⎥  ⎢              ⎥⎟
⎜⎢         2       ⎥  ⎢       2      ⎥⎟
⎜⎢ ⅈ⋅s    s        ⎥  ⎢ⅈ⋅s  -s       ⎥⎟
⎜⎢ ───    ──     1 ⎥, ⎢───  ────   0 ⎥⎟
⎜⎢  2     2        ⎥  ⎢ 2    2       ⎥⎟
⎜⎢                 ⎥  ⎢              ⎥⎟
⎜⎢-ⅈ⋅s             ⎥  ⎢ⅈ⋅s           ⎥⎟
⎜⎢─────    1     0 ⎥  ⎢───   0     0 ⎥⎟
⎝⎣  2              ⎦  ⎣ 2            ⎦⎠

In [ ]:
# U, coshr = E.diagonalize()
# (U,coshr)
lamda = symbols('lamda')
p = E.charpoly(lamda)
q = F.charpoly(lamda)
epoly = factor(simplify(expand(p.as_expr())))
fpoly = factor(simplify(expand(q.as_expr())))
(epoly,fpoly)

⎛   3      2  2      2          4      2         3      2  2        2    4⎞
⎜8⋅λ  - 4⋅λ ⋅s  - 8⋅λ  - 8⋅λ + s  + 4⋅s  + 8  8⋅λ  + 4⋅λ ⋅s  + 4⋅λ⋅s  + s ⎟
⎜───────────────────────────────────────────, ────────────────────────────⎟
⎝                     8                                    8              ⎠

In [ ]:
re = (Dagger(E)*E)
r = re.charpoly(lamda)
rpoly = factor(simplify(expand(r.as_expr())))
rpoly
# solve(rpoly)
# G = simplify(E.subs(s,0))
# U,S,V = E.singular_value_decomposition()

 ⎛      3       2  4       2  2        2        6         4          2        
-⎝- 64⋅λ  + 16⋅λ ⋅s  + 64⋅λ ⋅s  + 192⋅λ  - 8⋅λ⋅s  - 48⋅λ⋅s  - 128⋅λ⋅s  - 192⋅λ
──────────────────────────────────────────────────────────────────────────────
                                                        64                    

    8      6       4       2     ⎞ 
 + s  + 8⋅s  + 32⋅s  + 64⋅s  + 64⎠ 
───────────────────────────────────
                                   

In [ ]:
a,b,c,d = symbols('a b c d')
a=1
b=-1
c=-(s**2 +1)
d = 1
delta_0 = simplify((b**2 -3*a*c))
delta_1 = simplify((2*b**3 - 9*a*b*c+27*a*a*d))
# simplify(expand(delta_0**3))
u = simplify(sqrt(delta_1**2 - 4 *delta_0**3))
v = simplify(expand((delta_1 + u)/2))
C = cbrt(v)
(C,delta_0)

⎛      ____________________________________________          ⎞
⎜     ╱                 ______________________               ⎟
⎜    ╱       2         ╱       4       2                     ⎟
⎜   ╱     9⋅s    3⋅s⋅╲╱  - 12⋅s  - 39⋅s  - 96           2    ⎟
⎜3 ╱    - ──── + ───────────────────────────── + 8 , 3⋅s  + 4⎟
⎝╲╱        2                   2                             ⎠

In [ ]:
p,q,r= symbols('p q r')
q = -9/2*s**2 + 8
p = 12*s**4 + 39*s**2 + 96
r = 3*s/2*sqrt(p)

simplify( cbrt(factor(q**2 + r**2)))

(expand(q**2 - r**2)),simplify(2*q*r)


⎛                                                         ____________________
⎜      6         4          2         ⎛             2⎞   ╱     4       2      
⎝- 27⋅s  - 67.5⋅s  - 288.0⋅s  + 64, s⋅⎝24.0 - 13.5⋅s ⎠⋅╲╱  12⋅s  + 39⋅s  + 96 

⎞
⎟
⎠